In [ ]:
import asyncio
import os
import json
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client


In [ ]:
# Test Filesystem MCP Server
async def test_filesystem_server():
    current_dir = os.getcwd()
    
    # Create test directory for filesystem operations
    test_dir = os.path.join(current_dir, "test-filesystem")
    os.makedirs(test_dir, exist_ok=True)
    print(f"📁 Test directory: {test_dir}\n")

    os.makedirs(os.path.join(test_dir, "reports"), exist_ok=True)
    
    # Connect to Filesystem MCP server
    # This server operates on the test-filesystem directory
    server_params = StdioServerParameters(
        command="npx",
        args=["-y", "@modelcontextprotocol/server-filesystem", test_dir]
    )
    
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            
            # List available tools
            print("📋 Available Filesystem Tools:")
            tools_response = await session.list_tools()
            for tool in tools_response.tools:
                print(f"  - {tool.name}: {tool.description}")
            
            print("\n💾 Testing Filesystem Operations:")
            
            # Test 1: Write a text file
            print("\n1. Writing text file...")
            result1 = await session.call_tool("write_file", {
                "path": "test-filesystem/test.txt",
                "content": "Hello from Filesystem MCP Server!\nThis is a test file."
            })
            print(f"   Result: {result1.content[0].text if result1.content else 'Success'}")
            
            # Verify file exists
            file_path = os.path.join(test_dir, "test.txt")
            if os.path.exists(file_path):
                print(f"   ✅ File created: {file_path}")
            
            # Test 2: Read the file back
            print("\n2. Reading text file...")
            result2 = await session.call_tool("read_file", {
                "path": "test-filesystem/test.txt"
            })
            print(f"   Content: {result2.content[0].text[:100] if result2.content else 'No content'}")
            
            # Test 3: Write a JSON file (like a test report)
            print("\n3. Writing JSON report...")
            test_report = {
                "feature": "Todo App Test",
                "scenarios": [
                    {
                        "name": "Add todo item",
                        "status": "PASSED",
                        "steps": 5
                    },
                    {
                        "name": "Mark complete",
                        "status": "PASSED",
                        "steps": 3
                    }
                ],
                "summary": {
                    "total": 2,
                    "passed": 2,
                    "failed": 0
                }
            }
            
            result3 = await session.call_tool("write_file", {
                "path": "test-filesystem/reports/test_report.json",
                "content": json.dumps(test_report, indent=2)
            })
            print(f"   Result: {result3.content[0].text if result3.content else 'Success'}")
            
            # Test 4: List directory contents
            print("\n4. Listing directory contents...")
            result4 = await session.call_tool("list_directory", {
                "path": "test-filesystem"
            })
            print(f"   Files/Folders:")
            for item in result4.content[0].text.split('\n') if result4.content else []:
                if item.strip():
                    print(f"     - {item}")
            
            # Test 5: Read the JSON report back
            print("\n5. Reading JSON report...")
            result5 = await session.call_tool("read_file", {
                "path": "test-filesystem/reports/test_report.json"
            })
            if result5.content:
                report_data = json.loads(result5.content[0].text)
                print(f"   Report Feature: {report_data['feature']}")
                print(f"   Total Scenarios: {report_data['summary']['total']}")
                print(f"   Passed: {report_data['summary']['passed']}")
            
            # Test 6: Create multiple files (like screenshots metadata)
            os.makedirs(os.path.join(test_dir, "screenshots"), exist_ok=True)
            print("\n6. Writing multiple files...")
            for i in range(3):
                result = await session.call_tool("write_file", {
                    "path": f"test-filesystem/screenshots/screenshot_{i}.txt",
                    "content": f"Screenshot {i} metadata\nTimestamp: 2024-10-22\nTest: scenario_{i}"
                })
            print(f"   ✅ Created 3 screenshot metadata files")
            
            # Test 7: List screenshots directory
            print("\n7. Listing screenshots directory...")
            result7 = await session.call_tool("list_directory", {
                "path": "test-filesystem/screenshots"
            })
            print(f"   Screenshots:")
            for item in result7.content[0].text.split('\n') if result7.content else []:
                if item.strip():
                    print(f"     - {item}")
            
            # Test 8: Get directory info (if available)
            print("\n8. Getting directory info...")
            result8 = await session.call_tool("get_file_info", {
                "path": "test-filesystem/test.txt"
            }) if "get_file_info" in [t.name for t in tools_response.tools] else None
            
            if result8:
                print(f"   File info: {result8.content[0].text if result8.content else 'No info'}")
            else:
                print(f"   (get_file_info not available)")
            
            print("\n✅ All Filesystem tests completed!")
            print(f"\n📂 Check test files in: {test_dir}")
            print(f"\nDirectory structure:")
            for root, dirs, files in os.walk(test_dir):
                level = root.replace(test_dir, '').count(os.sep)
                indent = ' ' * 2 * level
                print(f"{indent}{os.path.basename(root)}/")
                subindent = ' ' * 2 * (level + 1)
                for file in files:
                    print(f"{subindent}{file}")


In [ ]:
await test_filesystem_server()